# Collect info from all connectivity jsons

In [ ]:
import os
import glob
import pandas as pd
import ephys_analysis.funcs_sorting as sort
import json
import numpy as np
import ephys_analysis.funcs_for_results_tables as get_results
import ephys_analysis.funcs_plotting_raw_traces as funcs_plotting_raw_traces
import PyQt5

def repatch_duplicates(df, ID_column):
    not_repatched_cells = []
    for cell in df[ID_column].unique():
        if len(df[df[ID_column] == cell]) < 2:
            print('No repatch partner for ' + cell)
            not_repatched_cells.append(cell)

    duplicates = []
    for cell in df[ID_column].unique():
        if len(df[df[ID_column] == cell]) > 2:
            print('Repeated con_IDs' + cell)
            duplicates.append(cell)
    return not_repatched_cells, duplicates

In [ ]:
# update to the latest df_intr, if needed
df_intr = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/intrinsic_properties/2024-11-27_collected.xlsx')
df_intr = df_intr.sort_values('OP')
df_intr = df_intr[df_intr.hrs_incubation != 0]

# create OP and age dictionary, to cross check info 
op_age_dict, op_hrs_incubation = {}, {}
for OP in df_intr.OP.unique():
    op_age_dict[OP] = float(np.mean(df_intr.patient_age[df_intr.OP == OP]))
    op_hrs_incubation[OP] = float(np.mean(df_intr.hrs_incubation[df_intr.OP == OP]))

In [ ]:
# missing in the intrinsic dataframe
ops_add = ['OP231109', 'OP230704', 'OP220413', 'OP240516', 'OP220615', 'OP230417', 'OP230504']
ages_add = [44, 16, 17, 27, 14, 4.5, 56]
hrs_inc_add = [24, 3, 20, 0, 0, 0, 3]

for g, op in enumerate(ops_add):
    op_age_dict[op] = ages_add[g]
    op_hrs_incubation[op] = hrs_inc_add[g]

In [ ]:
HUMAN_DIR = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/data/human/'
PATHS = [HUMAN_DIR + 'data_verji/', HUMAN_DIR + 'data_rosie/']

In [ ]:
# collect all con_screen metadata
OPs, ages, patchers, fns, days, hrs_incubation, slices, treatments, pre_chans, post_chans = [], [], [], [], [], [], [], [], [], []
for dir in PATHS:
    op_dirs = [d for d in os.listdir(dir) if d.startswith('OP')]
    if 'rosie' in dir:
        patcher = 'Rosie'
    else:
        patcher = 'Verji'
    for op_dir in op_dirs:
        # lab_book_paths = glob.glob(f"{dir + op_dir}/*.xlsx")
        # lab_book = pd.read_excel([f for f in lab_book_path2 if not f.startswith(f"{dir + op_dir}/~$")][0])
        con_screen_jsons = glob.glob(f"{dir + op_dir}/*con_screen_only.json")
        if len(con_screen_jsons) < 1:
            print('no con_screen_json for ' + op_dir)
            continue
        else:
            con_screen_json = json.load(open(con_screen_jsons[0]))[0]
        file_list = sort.get_abf_files(sorted(os.listdir(dir + op_dir)))
        for i, slic in enumerate(con_screen_json['slices']):
            if 'pre_chans' not in con_screen_json.keys():
                print('no connections for OP' + con_screen_json['OP_time'][:-6])
                continue
            if len(con_screen_json['pre_chans']) == 0:
                print('no connections for OP' + con_screen_json['OP_time'][:-6])
            else:
                for j, pre_chan in enumerate(con_screen_json['pre_chans'][i]):
                    OP = 'OP' + con_screen_json['OP_time'][:-6]
                    OPs.append(OP)
                    ages.append(op_age_dict[OP])
                    fns.append(file_list[con_screen_json['con_screen_file'][i]])
                    if len(slic) > 2:
                        days.append('D2')
                        hrs_incubation.append(op_hrs_incubation[OP])
                    else:
                        days.append('D1')
                        hrs_incubation.append(0)
                    slices.append(slic)
                    treatments.append(con_screen_json['treatment'][i])
                    pre_chans.append(pre_chan)
                    post_chans.append(con_screen_json['post_chans'][i][j])
                    patchers.append(patcher)     

In [ ]:
# create and save dataframe
con_screen_df = pd.DataFrame({'OP' :OPs, 'patient_age': ages, 'patcher' : patchers, 'filename': fns, \
                              'day': days, 'hrs_incubation': hrs_incubation, 'slice': slices, \
                                'treatment': treatments, 'pre_chans': pre_chans, 'post_chans': post_chans })

del OPs, ages, fns, days, hrs_incubation, slices, treatments, pre_chans, post_chans

In [ ]:
# con_screen_df.to_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/con_screen_collected.xlsx')

### Same for the intrinsic properties

In [ ]:
# collect all intrinsic metadata
OPs, ages, fns, days, hrs_incubation, slices, treatments, active_chans, patchers = [], [], [], [], [], [], [], [], []

OP_no_connectivity = ['OP231207', 'OP240208', 'OP231005', 'OP230914', 'OP230126', 'OP221116', 'OP210304', 'OP201020']
for dir in PATHS:
    if 'rosie' in dir:
        patcher = 'Rosie'
    else:
        patcher = 'Verji'
    op_dirs = [d for d in os.listdir(dir) if d.startswith('OP')]
    for op_dir in op_dirs:
        if op_dir in OP_no_connectivity:
            continue
        meta_ = glob.glob(f"{dir + op_dir}/*meta_active_chans.json")
        if len(meta_) < 1:
            print('no meta_active_chans for ' + op_dir)
            continue
        else:
            meta_active_chans = json.load(open(meta_[0]))[0]
        file_list = sort.get_abf_files(sorted(os.listdir(dir + op_dir)))
        for i, slic in enumerate(meta_active_chans['slices']):
            # if 'active_chans' not in meta_active_chans.keys():
            #     print('no recordings for OP' + op_dir)
            #     continue
            # if len(meta_active_chans['active_chans']) == 0:
            #     print('no recordings for OP' + op_dir)
            for j, chan in enumerate(meta_active_chans['active_chans'][i]):
                OP = op_dir
                OPs.append(OP)
                ages.append(op_age_dict[OP])
                fns.append(file_list[meta_active_chans['vc_files'][i]])
                if len(slic) > 2:
                    days.append('D2')
                    hrs_incubation.append(op_hrs_incubation[OP])
                else:
                    days.append('D1')
                    hrs_incubation.append(0)
                slices.append(slic)
                treatments.append(meta_active_chans['treatment'][i])
                active_chans.append(chan)
                patchers.append(patcher)

In [ ]:
all_active_chans_df = pd.DataFrame({'filename': fns, 'patcher': patchers,'OP' :OPs, 'patient_age': ages, \
                                    'day': days, 'hrs_incubation': hrs_incubation, \
                                      'slice': slices, 'treatment': treatments, 'chan': active_chans})

del OPs, ages, fns, days, hrs_incubation, slices, treatments, active_chans, patchers

In [ ]:
# make the treatment a string
# was a list in the older analysis versions
new_tr = []
for tr in all_active_chans_df.treatment:
    if isinstance(tr, list):
        new_tr.append(tr[0])
    else:
        new_tr.append(tr)
all_active_chans_df['treatment'] = new_tr

In [ ]:
save_dir = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/'
all_active_chans_df = all_active_chans_df.sort_values(['OP', 'filename', 'chan'])
all_active_chans_df.to_excel(save_dir + 'all_active_chans.xlsx')
print('df saved to be completed')

## Check repatched cells and connections

In [ ]:
# open the intrinsic meta json collected df
# df_all_cells = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/all_active_chans_filled_in.xlsx')

In [ ]:
# take repatch only df
df_all_repatched = df_all_cells[df_all_cells['repatch'] == 'yes']
df_all_repatched.reset_index(inplace = True, drop = True)
patcher_dict = {'Verji': 'vm',
                'Rosie': 'rs'}

In [ ]:
# create cell IDs
cell_IDs = []
for i, op in enumerate(df_all_repatched.OP):
    initials = patcher_dict[df_all_repatched.patcher[i]]

    if df_all_repatched.day[i] == 'D1':
        slic = df_all_repatched.slice[i]
        ch = str(int(df_all_repatched.chan[i]))
    else:
        slic = df_all_repatched.slice[i][:2]
        ch = str(int(df_all_repatched.repatched_ch_from_D1[i]))
        
    cell_IDs.append(initials + '_' + op[2:] + '_'  + slic + '_' + ch)
df_all_repatched['cell_IDs'] = cell_IDs

In [ ]:
print('num unique cell_IDs: ' + str(len(df_all_repatched['cell_IDs'].unique())))
print('all repatched cells : ' + str(len(df_all_repatched)))

In [ ]:
# check for not repatched cells and duplicates 
not_repatched_cells, duplicates = repatch_duplicates(df_all_repatched, 'cell_IDs')

In [ ]:
# df_all_cells.to_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/all_active_chans_filled_in.xlsx')

## Compare con_screen and intrinsic to find repatched connections

In [ ]:
con_screen_df = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/con_screen_collected.xlsx')

In [ ]:
patcher_dict = {'Verji': 'vm',
                'Rosie': 'rs'}
con_screen_df.columns

In [ ]:
# create pre and post cell IDs
pre_cell_IDs, post_cell_IDs = [], []
for i, op in enumerate(con_screen_df.OP):
    initials = patcher_dict[con_screen_df.patcher[i]]
    slic = con_screen_df.slice[i]
    pre_chan = con_screen_df.pre_chans[i]
    post_chan = con_screen_df.post_chans[i]
    pre_cell_IDs.append(initials + '_' + op[2:] + '_'  + slic + '_' + str(pre_chan))
    post_cell_IDs.append(initials + '_' + op[2:] + '_'  + slic + '_' + str(post_chan))

con_screen_df['pre_chan_IDs'] = pre_cell_IDs
con_screen_df['post_chan_IDs'] = post_cell_IDs

In [ ]:
# create temp cell_IDs repatched cells
temp_IDs = []
for k, op in enumerate(df_all_repatched.OP):
    initials = patcher_dict[df_all_repatched.patcher[k]]
    slic = df_all_repatched.slice[k]
    ch = str(int(df_all_repatched.chan[k]))
    temp_IDs.append(initials + '_' + op[2:] + '_'  + slic + '_' + ch)

# Find duplicates
seen = set()
duplicates = []

for item in temp_IDs:
    if item in seen:
        duplicates.append(item)
    else:
        seen.add(item)

# Print the list of duplicates
print(f"Duplicates: {duplicates}") # if none --> continue
df_all_repatched['temp_IDs'] = temp_IDs 

In [ ]:
repatch = []
for g, pre_cell in enumerate(pre_cell_IDs):
    post_cell = post_cell_IDs[g]
    if pre_cell in temp_IDs and post_cell in temp_IDs:
        repatch.append('yes')
    else:
        repatch.append('no')
con_screen_df['repatch'] = repatch

print('num repatched connections ' + str(repatch.count('yes')))
print('num not repatched connections ' + str(repatch.count('no')))

In [ ]:
repatched_cons_df = con_screen_df[con_screen_df['repatch'] == 'yes']
repatched_cons_df.reset_index(inplace = True)

In [ ]:
con_IDs, pre_IDs_repatch, post_IDs_repatch = [], [], []
for m, pre_cell in enumerate(repatched_cons_df.pre_chan_IDs):
    post_cell = repatched_cons_df.post_chan_IDs[m]
    
    cell_ID_pre = df_all_repatched['cell_IDs'].loc[df_all_repatched.temp_IDs == pre_cell].values[0]
    cell_ID_post = df_all_repatched['cell_IDs'].loc[df_all_repatched.temp_IDs == post_cell].values[0]

    pre_IDs_repatch.append(cell_ID_pre)
    post_IDs_repatch.append(cell_ID_post)
    con_IDs.append(cell_ID_pre + 'to' + cell_ID_post[-1])

repatched_cons_df['connection_ID'] = con_IDs
repatched_cons_df['pre_chan_IDs'] = pre_IDs_repatch
repatched_cons_df['post_chan_IDs'] = post_IDs_repatch

In [ ]:
not_repatched_cells, duplicates = repatch_duplicates(repatched_cons_df, 'connection_ID')

# for con in not_repatched_connections:
#     repatched_cons_df = repatched_cons_df.drop(repatched_cons_df.index[repatched_cons_df['connection_ID'] == con])     
# repatched_cons_df.reset_index(inplace = True, drop = True)


In [ ]:
# plot each non-repeating connection
# fill in excel with 'new' or 'disappearing'

con = ''

df_part = repatched_cons_df.loc[repatched_cons_df['connection_ID'] == con]
fn = df_part.filename.values[0]
OP = df_part.OP.values[0]
patcher = df_part.patcher.values[0]
slic = df_part.slice.values[0]

work_dir, filenames, indices_dict, slice_names = sort.get_OP_metadata(HUMAN_DIR, OP, patcher, 'old')
con_screen_json = sort.get_json_meta_connect_all(HUMAN_DIR, OP, patcher)[0]

slic_indx_D1 = con_screen_json['slices'].index(slic)
try:
    slic_indx_D2 = con_screen_json['slices'].index(slic + '_D2')
except:
    slic_indx_D2 = con_screen_json['slices'].index(slic[:2])
active_chans_D1 = con_screen_json['active_chans'][slic_indx_D1]
active_chans_D2 = con_screen_json['active_chans'][slic_indx_D2]
fn_2 = filenames[con_screen_json['con_screen_file'][slic_indx_D2]]


%matplotlib qt
funcs_plotting_raw_traces.plot_con_screen_all(work_dir + fn, active_chans_D1)
funcs_plotting_raw_traces.plot_con_screen_all(work_dir + fn_2, active_chans_D2)
%matplotlib


In [ ]:
fn = '21d10035.abf'
work_dir = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/data/human/data_verji/OP211209/'

funcs_plotting_raw_traces.plot_con_screen_all(work_dir + fn, [1,2,4,7,8])

In [ ]:
# repatched_cons_df.to_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/repatched_connections_with_comments.xlsx')

# Compare collected to intrinsic

In [ ]:
repatched_cons_collected = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/repatched_connections_collected_from_analyzed_folders.xlsx')
collected_repatched = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/repatched_single_cells.xlsx')

In [ ]:
# temporary ID for repatched cells, to match the one from the connections df
temp_IDs_repatched_cells = []
for g, op in enumerate(collected_repatched.OP):
    slic = collected_repatched.slice.iloc[g]
    chan = str(int(collected_repatched.chan.iloc[g]))
    temp_IDs_repatched_cells.append(op[2:] + slic  + chan)

collected_repatched['temp_ID'] = temp_IDs_repatched_cells

In [ ]:
# create temp pre and post cell IDs
pre_cell_IDs, post_cell_IDs = [], []
for f, pre_chan in enumerate(repatched_cons_collected.chan_pre):
    op = repatched_cons_collected.OP.iloc[f][2:]
    slic = repatched_cons_collected.slice.iloc[f]
    post_chan = repatched_cons_collected.chan_post.iloc[f]

    pre_cell_IDs.append(op + slic + str(int(pre_chan)))
    post_cell_IDs.append(op + slic + str(int(post_chan)))

# checking for mistakes
repatches = []
for d, pre_cell in enumerate(pre_cell_IDs):
    post_cell = post_cell_IDs[d]
    if pre_cell in temp_IDs_repatched_cells and post_cell in temp_IDs_repatched_cells:
        repatches.append('yes')
    else:
        repatches.append('no')

print('Number of not repatched cells: ' + str(repatches.count('no')))

drop_indices = [index for index, value in enumerate(repatches) if value == 'no']

# drop from the list
pre_cell_IDs = [item for index, item in enumerate(pre_cell_IDs) if index not in drop_indices]
post_cell_IDs = [item for index, item in enumerate(post_cell_IDs) if index not in drop_indices]

# drop from the dataframe
repatched_cons_collected = repatched_cons_collected.drop(drop_indices)
repatched_cons_collected.reset_index(drop=True, inplace=True)

In [ ]:
# create true connection IDs
con_IDs_new = []
for p, pre_cell in enumerate(pre_cell_IDs):
    post_cell = post_cell_IDs[p]

    pre_cell_true_ID = collected_repatched['cell_IDs'][collected_repatched['temp_ID'] == pre_cell].values[0]
    post_cell_true_ID = collected_repatched['cell_IDs'][collected_repatched['temp_ID'] == post_cell].values[0][-1]

    # con_IDs_new.append()
    con_IDs_new.append(pre_cell_true_ID + 'to' + post_cell_true_ID[-1])

repatched_cons_collected['con_IDs_new'] = con_IDs_new


In [ ]:
# repatched_cons_collected.to_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/repatched_connections_collected_from_analyzed_folders.xlsx')

# Compare the two connections tables

In [ ]:
# all_connections_from_folders, all_cons_VC =  get_results.collect_connections_df()
# all_connections_from_folders = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/connectivity/2024-12-09_connectivity.xlsx')
# repatch_con_screen_from_tables = all_connections_from_folders[all_connections_from_folders['repatch'] == 'yes']
repatched_cons_collected = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/repatched_connections_with_comments.xlsx')
repatch_con_screen_from_tables = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/repatched_connections_collected_from_analyzed_folders.xlsx')

In [ ]:
repatched_cons_collected.reset_index(drop = True)
drop_indices = []
for h, comment in enumerate(repatched_cons_collected.comment):
    if isinstance(comment, float):
        continue
    if 'exclude' in comment:
        drop_indices.append(h)

for indx in drop_indices:
    repatched_cons_collected = repatched_cons_collected.drop(indx)     
repatched_cons_collected.reset_index(inplace = True, drop = True)


In [ ]:
not_repatched_cells, duplicates = repatch_duplicates(repatched_cons_collected, 'connection_ID')
for con in not_repatched_cells:
    print(repatched_cons_collected.comment[repatched_cons_collected.connection_ID == con])

In [ ]:
cons_from_tables = set(repatch_con_screen_from_tables.con_IDs_new)
cons_from_comparison_with_intrinsic = set(repatched_cons_collected.connection_ID)

in_tables_not_in_comparison = list(cons_from_tables - cons_from_comparison_with_intrinsic)
in_comparison_not_in_collected = list(cons_from_comparison_with_intrinsic - cons_from_tables)

In [ ]:
sorted(in_tables_not_in_comparison)

# get extra data

In [ ]:
data_to_add = pd.DataFrame()
for con in in_comparison_not_in_collected:
    con_data = repatched_cons_collected[repatched_cons_collected.connection_ID == con]
    data_to_add = pd.concat([data_to_add[:], con_data])


In [ ]:
# analyse extra set of data

HUMAN_DIR = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/data/human/'

con_data = pd.DataFrame(columns = ['OP', 'fn', 'slice', 'day', 'connection_ID', 'repatch', 'treatment', 
'hrs_after_OP', 'hrs_incubation','chan_pre', 'chan_post', 'Vm pre', 'Vm post', 
'Amp 1','Amp 2','Amp 3', 'Amp 4',	'Lat1',	'Lat2',	'Lat3',	'Lat4', 'num excluded swps', 'comments'])

exclude = ''
for q, fn in enumerate(data_to_add.filename):
    OP = data_to_add.OP.iloc[q]
    patcher = data_to_add.patcher.iloc[q]
    work_dir, filenames, indices_dict, slice_names = sort.get_OP_metadata(HUMAN_DIR, OP, patcher, 'old')
    con_screen_file = work_dir + fn

    pre_cell = data_to_add.pre_chans.iloc[q]
    post_cell = data_to_add.post_chans.iloc[q]

    pre_signal, es, vm_pre = con_param.presynaptic_screen(con_screen_file, pre_cell)
    post_signal, vm_post = con_param.postsynaptic_screen(con_screen_file, post_cell, es)
    if (np.array(vm_post)).size == 0:
        print('QC not passed!!')
        exclude = 'all'
        es2 = []
        post_signal, vm_post = con_param.postsynaptic_screen(con_screen_file, post_cell, es2)
        continue
    mean_pre, mean_post, pre_window, post_window, preAPs_shifted, preAPs = \
        con_param.get_analysis_window(pre_signal, post_signal)
    pre_signal, post_signal = con_param.remove_sweeps_with_POSTsynAPs(pre_signal, post_signal, preAPs)
    post_peaks = con_param.find_postsynaptic_peaks(post_window, preAPs_shifted)
    post_local_baseline = con_param.get_psp_baselines(post_window,preAPs_shifted)
    onsets = con_param.get_onsets(preAPs_shifted, post_window, post_peaks, post_local_baseline)
    latency = con_param.latencies(onsets, preAPs_shifted)
    amps = con_param.get_amps(post_peaks, post_local_baseline)

    df_add = pd.DataFrame({'OP': OP, 'fn': fn, 'slice': slic, 'day':data_to_add.day.iloc[q], 
                           'treatment': data_to_add.treatment.iloc[q], 
    'connection_ID' : data_to_add.connection_ID.iloc[q]	,
    'chan_pre': pre_cell, 'chan_post': post_cell, 'Vm pre' :vm_pre, 'Vm post': vm_post,
    'Amp 1': amps[0], 'Amp 2': amps[1],	'Amp 3': amps[2],	'Amp 4': amps[3],	
        'Lat1': latency[0][0],	'Lat2' : latency[1][0],	'Lat3': latency[2][0],	'Lat4': latency[3][0], 
    'num excluded swps': len(es), 'comments': exclude}, index=[0])
    con_data = pd.concat([con_data.loc[:], df_add]).reset_index(drop=True)

    funcs_plotting_raw_traces.plot_connection_window(con_screen_file, pre_cell, post_cell, pre_window, \
            post_window, preAPs_shifted, post_signal, onsets, preAPs, post_peaks, post_local_baseline)
    funcs_plotting_raw_traces.plot_post_cell(con_screen_file, pre_cell, post_cell)

In [ ]:
# add data to df
repatch_con_screen_from_tables = pd.concat([repatch_con_screen_from_tables.loc[:],con_data.loc[:]]).reset_index(drop=True)
repatch_con_screen_from_tables = repatch_con_screen_from_tables.drop_duplicates()


In [ ]:
# compare_hrs_incubation
df = repatched_cons_collected # repatched_cons_collected

OP_hrs_inc_dict = {}
for OP in repatched_cons_collected.OP.unique():
    hrs_inc = repatched_cons_collected.hrs_incubation[(repatched_cons_collected.OP == OP) & \
                                                            (repatched_cons_collected.hrs_incubation != 0)]
    if len(hrs_inc) == 0:
        OP_hrs_inc_dict[OP] = ''
    else:
        OP_hrs_inc_dict[OP] = int(hrs_inc.values[0])


In [ ]:
# repatch_con_screen_from_tables.to_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/lab_seminar_Dec_24/all_analyzed_connection_complete.xlsx')

# Plot results

In [ ]:
import ephys_analysis.funcs_plot_intrinsic_props as pl_intr
import pandas as pd

def QC_connectivity_df(df):
    mask = (df['Vm pre'] < -40 ) & (df['Vm post'] < -40)
        # (df['Amp 1'] > 0) &  (df['Amp 2'] > 0 )   & \
        #     (df['Amp 3'] > 0 ) &  (df['Amp 4'] > 0)
    df = df.loc[mask, :]
    return df

DEST_DIR = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/lab_seminar_Dec_24/'
con_screen_df = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/lab_seminar_Dec_24/all_repatched_connection_complete.xlsx')

In [ ]:
# only adults
con_screen_df = con_screen_df[con_screen_df.patient_age > 12]
con_screen_df = con_screen_df[(con_screen_df.hrs_incubation == 0) |
                                (con_screen_df.hrs_incubation > 15)].copy()
repa, duplic = repatch_duplicates(con_screen_df, 'con_IDs_new')

for con in repa:
    drop_indx = con_screen_df[con_screen_df['con_IDs_new'] == con].index
    con_screen_cleaned = con_screen_df.drop(drop_indx)
con_screen_cleaned.reset_index(inplace = True)
con_screen_df = con_screen_cleaned
con_screen_df

In [ ]:
op_color_dict = pl_intr.get_op_color_dict(con_screen_df)

# con_screen_df['Amp 1'] = con_screen_df['Amp 1'].fillna(0)
# con_screen_df['Amp 2'] = con_screen_df['Amp 2'].fillna(0)
# con_screen_df['Amp 3'] = con_screen_df['Amp 3'].fillna(0)
# con_screen_df['Amp 4'] = con_screen_df['Amp 4'].fillna(0)
# con_screen_df.columns[con_screen_df.isna().any()].tolist()

connect_QC_passed = QC_connectivity_df(con_screen_df)

repatch_connect = pl_intr.get_repatch_connectivity_df(connect_QC_passed)
# how to deal with disappearing connections
repatch_connect.fillna(0, inplace = True) #fill missing values with 0
pl_intr.plot_connect_amplitude(repatch_connect, 'repatch_IC', \
    op_color_dict ,'con_IDs_new' ,'amp', DEST_DIR)

### collect slice connections data

In [ ]:
all_cons_IC, all_cons_VC =  get_results.collect_connections_df()
all_cons_IC['Amp 1'] = all_cons_IC['Amp 1'].fillna(0)
all_cons_IC['Amp 2'] = all_cons_IC['Amp 2'].fillna(0)
all_cons_IC['Amp 3'] = all_cons_IC['Amp 3'].fillna(0)
all_cons_IC['Amp 4'] = all_cons_IC['Amp 4'].fillna(0)
all_cons_IC.columns[all_cons_IC.isna().any()].tolist()

In [ ]:
all_cons_IC = all_cons_IC[all_cons_IC.OP != 'OP201020'] 

In [ ]:
patient_age = []
for i, op in enumerate(all_cons_IC.OP):
    patient_age.append(op_age_dict[op])

all_cons_IC['patient_age'] = patient_age 
all_cons_IC = all_cons_IC[all_cons_IC['patient_age'] > 12]

all_cons_IC.hrs_incubation.astype(int)
all_cons_IC = all_cons_IC[(all_cons_IC.hrs_incubation == 0) |
                                (all_cons_IC.hrs_incubation > 15)].copy()

all_cons_IC.hrs_incubation.describe()

In [ ]:
connect_QC_passed = pl_intr.get_QC_connectivity_df(all_cons_IC)
op_color_dict = pl_intr.get_op_color_dict(all_cons_IC)
connect_slice = connect_QC_passed[connect_QC_passed['repatch'] == 'no']
pl_intr.plot_connect_amplitude(df = connect_slice, data_type = 'all_IC', \
    op_color_dict = op_color_dict , con_ID_col = 'connection_ID' ,results_ ='amp', \
        destination_dir =DEST_DIR)

In [ ]:
# connect_slice.to_csv('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/lab_seminar_Dec_24/for_R/slice_connections_adult.csv')
# repatch_connect.to_csv('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/lab_seminar_Dec_24/for_R/repatch_connections_adult.csv')
